#Random Forest(랜덤 포레스트)
##- 결정 트리의 단점인 오버피팅 문제를 완하시켜주는 발전된 형태의 트리 모델
##- 여러 모델을 활용하여 하나의 모델을 이루는 기법을 앙상블이라 부름
##- 랜덤으로 독립적인 트리를 여러 개 만들어서 결정 트리의 오버피팅 문제를 완하해 분류하는 알고리즘

#장점 :
##1. 아웃라이어에 거의 영향을 받지 않음
##- 결정 트리와 마찬가지임
##2. 별다른 가정 없이 잘 작동함
##- 선형/비선형 데이터에 상관없이 잘 작동

#단점 :
##1. 학습 속도가 상대적으로 느림
##2. 모델에 대한 해석이 어려움
##- 수많은 트리를 동원하기 때문에 모델에 대한 해석이 어려

##1단계 : 문제정의
##2단계 : 라이브러리 및 데이터 불러오기, 데이터 확인하기
##3단계 : 전처리
##ㄴ텍스트 데이터
##ㄴ결측치 처리와 더미 변스 변환
##4단계 : 모델링 예측하기
##5단계 : 이해하기
##ㄴK겹 교차검증
##ㄴ랜덤 포레스트
##6단계 : 하이퍼 파라미터 튜닝


#목표 : 중고차 판매 이력 데이터셋을 이용해 중고차 가격을 예측
#데이터 : car.csv
#종속변수 : selling_price(판매 가격)
#평가지표 : RMSE
#모델 : FandomForestRegressor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import kagglehub
import pandas as pd
import os

# 1. 데이터셋 다운로드
path = kagglehub.dataset_download("sajaabdalaal/car-details-v3csv")

print("Path to dataset files:", path)

# 2. 다운로드된 경로 확인
print("Path to dataset files:", path)

# 3. 파일 목록 확인
print("Files in directory:", os.listdir(path))

100%|██████████| 168k/168k [00:00<00:00, 22.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sajaabdalaal/car-details-v3csv/versions/1
Path to dataset files: /root/.cache/kagglehub/datasets/sajaabdalaal/car-details-v3csv/versions/1
Files in directory: ['Car details v3.csv']


In [3]:
# 4. CSV 파일 읽기
csv_file = os.path.join(path, "Car details v3.csv")
df = pd.read_csv(csv_file, encoding='latin1')  # 이 데이터는 latin1로 인코딩된 경우가 많음

# 5. 데이터 출력
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [4]:
data = df.copy()
df.info()
#결측치 존재
#engine, max_power에는 단위가 함께 입력되어 있기 때문에
#단위는 제거하고 숫자 형태로 저장되도록 전처리 실행 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [5]:
round(data.describe(),2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


In [6]:
#텍스트 제거, 숫자형 변환 전처리
data['engine'].str.split(expand = True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [7]:
data[['engine','engine_unit']] = data['engine'].str.split(expand=True)

In [8]:
data['engine'].head()#데이터 형태가 object다

,engine
0,1248
1,1498
2,1497
3,1396
4,1298


In [9]:
data['engine_unit'].head()

,engine_unit
0,CC
1,CC
2,CC
3,CC
4,CC


In [10]:
data['engine'] = data['engine'].astype('float32')

In [11]:
data['engine'].head()

,engine
0,1248.0
1,1498.0
2,1497.0
3,1396.0
4,1298.0


In [12]:
data['engine_unit'].unique()
#CC아니면 단위가 작성이 안되어서 분리가 되지 않은 데이터가 존재했

array(['CC', nan], dtype=object)

In [13]:
data.drop('engine_unit',axis = 1, inplace = True)

In [14]:
data['max_power'].head()
#bhp 분리

,max_power
0,74 bhp
1,103.52 bhp
2,78 bhp
3,90 bhp
4,88.2 bhp


In [15]:
data[['max_power','max_power_unit']] = data['max_power'].str.split(expand=True)

In [16]:
data['max_power'].head()

,max_power
0,74
1,103.52
2,78
3,90
4,88.2


In [17]:
data['max_power'] = data['max_power'].astype('float32')
#bhp가 제거되지 않은 데이터가 존재하는듯?

ValueError: could not convert string to float: 'bhp'

In [18]:
data[data['max_power'] == 'bhp']
#숫자 없이 bhp만 존재하는 데이터가 존재
#split이 되지 않았나보다
#이 데이터만 제거하거나 에러처리

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [19]:
def isFloat(value):
  try:
    num = float(value)#데이터를 float형으로 바꿔라
    return num
  except ValueError:#ValueError가 뜨면
    return np.nan #결측치로 처리

In [20]:
data['max_power'] = data['max_power'].apply(isFloat)

In [21]:
data['max_power'].head()

,max_power
0,74.00
1,103.52
2,78.00
3,90.00
4,88.20


In [22]:
data['max_power_unit'].unique()
#bhp말고는 다 단위가 입력 안된 데이터 뿐.. 필요없겠다

array(['bhp', nan, None], dtype=object)

In [23]:
data.drop('max_power_unit',axis = 1, inplace = True)

In [24]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74.00,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497.0,78.00,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396.0,90.00,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298.0,88.20,"11.5@ 4,500(kgm@ rpm)",5.0


In [25]:
data[['mileage','mileage_unit']] = data['mileage'].str.split(expand=True)

In [26]:
data['mileage'] = data['mileage'].astype('float32')

In [27]:
data['mileage_unit'].unique()
#단위가 여러 개 존재.. 데이터를 모두 같은 단위의 숫자로 볼 수 없다
#단위를 맞춰줘야 데이터 분석이 가능하다..
#연비 데이터(mileage)의 단위를 맞춰주기 위해선 연료(fuel)데이터의 단위도 중요하다
#따라서 연료 데이터를 확인하겠다

array(['kmpl', 'km/kg', nan], dtype=object)

In [28]:
data['fuel'].unique()
#4가지 연료가 있다..
#다른 종류의 연료로 주행거리를 비교하려면 같은 기준을 세워야 한다
#1달러 당 몇 키로를 주행할 수 있는지를 비교하여 연료의 종류와 상관없이
#동등한 비교가 될 수 있도록 단위를 환산

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [29]:
#Petrol : 리터당 $80.43
#Diesel : 리터당 $73.56
#LPG : 킬로그램당 $40.85
#CNG : 킬로그램당 $44.23
#라고 가정..(시세에 따라 계속 달라질 )

In [30]:
#1$당 주행거리를 환산해주는 함수 작성
#연비계산 함수
def mile(x):
  if x['fuel'] == 'Petrol':
    return x['mileage']/80.43
  elif x['fuel'] == 'Diesel':
    return x['mileage']/73.56
  elif x['fuel'] == 'LPG':
    return x['mileage']/40.85
  else:
    return x['mileage']/44.23

In [31]:
data['mileage'] = data.apply(mile,axis = 1)

In [32]:
data.drop('mileage_unit',axis=1,inplace=True)

In [33]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,"11.5@ 4,500(kgm@ rpm)",5.0


#랜덤 포레스트
##- 여러 트리를 활용하여 최종 모델을 만듦
##- 다양한 트리의 의견 반영 -> 오버피팅 위험을 낮출 수 있음
##- 랜덤 포레스트가 여러 개의 트리를 만들 때는 데이터 전체를 사용하지 않고, 매번 다른 일부의 데이터를 사용하여 다른 트리를 만들어냄
##- 각 트리에서 주어진 모든 독립변수를 사용하지 않고 일부 변수들만을 매번 다르게 추출하여 사용함
##- 특정 변수의 강력한 힘을 제어하고 오버피팅을 피할 수 있음

In [34]:
data['torque'].head()#자동차 힘(마력)

,torque
0,190Nm@ 2000rpm
1,250Nm@ 1500-2500rpm
2,"12.7@ 2,700(kgm@ rpm)"
3,22.4 kgm at 1750-2750rpm
4,"11.5@ 4,500(kgm@ rpm)"


In [35]:
data['torque'] = data['torque'].str.upper()

In [36]:
'ex' in 'example'

True

In [37]:
'ex' in 'university'

False

In [38]:
def torque_unit(x):
  if 'NM' in str(x):
    return 'Nm'
  elif 'KGM' in str(x):
    return 'kgm'

In [39]:
data['torque_unit'] = data['torque'].apply(torque_unit)

In [40]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [41]:
data[data['torque_unit'].isna()]

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,torque_unit
13,Maruti Swift 1.3 VXi,2007,200000,80000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
31,Fiat Palio 1.2 ELX,2003,70000,50000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
78,Tata Indica DLS,2003,50000,70000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
87,Maruti Swift VDI BSIV W ABS,2015,475000,78000,Diesel,Dealer,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
119,Maruti Swift VDI BSIV,2010,300000,120000,Diesel,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7846,Toyota Qualis Fleet A3,2000,200000,100000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
7996,Hyundai Santro LS zipPlus,2000,140000,50000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
8009,Hyundai Santro Xing XS eRLX Euro III,2006,145000,80000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
8068,Ford Figo Aspire Facelift,2017,580000,165000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None


In [42]:
data[data['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [43]:
data['torque_unit'].fillna('Nm',inplace=True)
#Nm과 kgm의 단위가 다르다
#단위의 사전지식이 있다는 가정 하에 진행
#일단 Nm으로 채움

<ipython-input-43-e3a1c98b625e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['torque_unit'].fillna('Nm',inplace=True)


In [44]:
#원하는 값만 분리하기
string_example = '12.7@ 2, 700(KGM@ RPOM)'

In [45]:
string_example[0]

'1'

In [46]:
string_example[4]

'@'

In [47]:
string_example[:4]

'12.7'

In [48]:
for i, j in enumerate(string_example):
  print(i, '번째 텍스트: ',j)

0 번째 텍스트:  1
1 번째 텍스트:  2
2 번째 텍스트:  .
3 번째 텍스트:  7
4 번째 텍스트:  @
5 번째 텍스트:   
6 번째 텍스트:  2
7 번째 텍스트:  ,
8 번째 텍스트:   
9 번째 텍스트:  7
10 번째 텍스트:  0
11 번째 텍스트:  0
12 번째 텍스트:  (
13 번째 텍스트:  K
14 번째 텍스트:  G
15 번째 텍스트:  M
16 번째 텍스트:  @
17 번째 텍스트:   
18 번째 텍스트:  R
19 번째 텍스트:  P
20 번째 텍스트:  O
21 번째 텍스트:  M
22 번째 텍스트:  )


In [49]:
for i, j in enumerate(string_example):
  if j not in '0123456789.':
    cut = i
    break


In [50]:
def split_num(x):
  x = str(x)
  for i, j in enumerate(x):
    if j not in '0123456789.':
      cut = i
      break
  return x[:cut]

In [51]:
data['torque'] = data['torque'].apply(split_num)

In [52]:
data['torque'].head()

,torque
0,190
1,250
2,12.7
3,22.4
4,11.5


In [53]:
data['torque'] = data['torque'].astype('float32')
#빈칸이 있는 데이터가 있다..

ValueError: could not convert string to float: ''

In [54]:
data['torque'].replace('',np.nan, inplace = True)
#빈칸을 nan으로 바꾸기

In [55]:
data['torque'] = data['torque'].astype('float32')

In [56]:
data['torque'].head()
#단위와 분리 완료

,torque
0,190.0
1,250.0
2,12.7
3,22.4
4,11.5


In [57]:
#단위에 따른 차이를 맞춰주는 과정 필요
#Nm = 9.8066 * kgm

In [58]:
def torque_trans(x):
  if x['torque_unit'] == 'kgm':
    return x['torque'] * 9.8066
  else :
    return x['torque']

In [59]:
data['torque'] = data.apply(torque_trans, axis = 1)

In [60]:
data.drop('torque_unit', axis = 1, inplace = True)

In [61]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.000000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.000000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.543818,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.667836,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.775900,5.0


#교차검증
##- 다양한 훈련셋/시험셋을 통하여 모델에 더 신뢰할 수 있는 평가를 하는 방법

#앙상블
##- 여러 모델을 만들고 각 예측값들을 투표/평균 등으로 통합하여 더 정확한 예측을 도모하는 방법

#랜덤 포레스트 주의사항
##- 랜덤 포레스트는 앙상블 기법을 사용한 트리 기반 모델 중 가장 보편적인 방법
##- 이후에 다루게 될 부스팅 모델에 비하면 예측력이나 속도에서 부족한 부분이 있음
##- 시각화 부분에서는 결정 트리에 못 미치나, 다음 단계인 부스팅 모델을 이해하려면 꼭 알아야 할 필수 알고리즘

In [62]:
data['name'] = data['name'].str.split(expand=True)[0]

In [63]:
data['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [64]:
#Land Rover
data['name'] = data['name'].replace('Land' , "Land Rover")

In [69]:
data.isna().mean()
#mileage, engine, seats는 같은 데이터에서 결측치가 있는듯함
#한 데이터에 결측치가 여러 개 존재한다는 뜻 -> 결측치를 채워도 해당 데이터는 모델 학습에 악영향을 끼칠 확률이 높다

,0
name,0.000000
year,0.000000
selling_price,0.000000
km_driven,0.000000
fuel,0.000000
seller_type,0.000000
transmission,0.000000
owner,0.000000
mileage,0.027190
engine,0.027190


In [70]:
data.dropna(inplace=True)
len(data)

7906

In [72]:
data = pd.get_dummies(data, columns = ['name','fuel','seller_type','transmission','owner'], drop_first = True)

In [73]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
1,2014,370000,120000,0.287384,1498.0,103.52,250.000000,5.0,False,False,...,True,False,False,True,False,True,False,True,False,False
2,2006,158000,140000,0.220067,1497.0,78.00,124.543818,5.0,False,False,...,False,False,True,True,False,True,False,False,False,True
3,2010,225000,127000,0.312670,1396.0,90.00,219.667836,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
4,2007,130000,120000,0.200174,1298.0,88.20,112.775900,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False


In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis = 1), data['selling_price'], test_size = 0.2, random_state = 100)

In [75]:
from sklearn.ensemble import RandomForestRegressor

In [76]:
model = RandomForestRegressor(random_state = 100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [77]:
from sklearn.metrics import mean_squared_error
print("train_rmse", mean_squared_error(y_train, train_pred)**0.5)
print("test_rmse: ",mean_squared_error(y_test, test_pred)**0.5)
#훈련셋의 에러는 작은편
#시험셋의 에러는 비교적 크다

train_rmse 53531.41548125947
test_rmse:  131855.18391308116


In [78]:
from sklearn.model_selection import KFold
#KFold 교차검증 실습

In [79]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
1,2014,370000,120000,0.287384,1498.0,103.52,250.000000,5.0,False,False,...,True,False,False,True,False,True,False,True,False,False
2,2006,158000,140000,0.220067,1497.0,78.00,124.543818,5.0,False,False,...,False,False,True,True,False,True,False,False,False,True
3,2010,225000,127000,0.312670,1396.0,90.00,219.667836,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
4,2007,130000,120000,0.200174,1298.0,88.20,112.775900,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,0.230014,1197.0,82.85,113.699997,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False
8124,2007,135000,119000,0.228385,1493.0,110.00,235.358400,5.0,False,False,...,True,False,False,True,False,True,True,False,False,False
8125,2009,382000,120000,0.262371,1248.0,73.90,190.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
8126,2013,290000,25000,0.320419,1396.0,70.00,140.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False


In [80]:
data.reset_index(drop=True, inplace = True)

In [81]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
1,2014,370000,120000,0.287384,1498.0,103.52,250.000000,5.0,False,False,...,True,False,False,True,False,True,False,True,False,False
2,2006,158000,140000,0.220067,1497.0,78.00,124.543818,5.0,False,False,...,False,False,True,True,False,True,False,False,False,True
3,2010,225000,127000,0.312670,1396.0,90.00,219.667836,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
4,2007,130000,120000,0.200174,1298.0,88.20,112.775900,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,82.85,113.699997,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False
7902,2007,135000,119000,0.228385,1493.0,110.00,235.358400,5.0,False,False,...,True,False,False,True,False,True,True,False,False,False
7903,2009,382000,120000,0.262371,1248.0,73.90,190.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
7904,2013,290000,25000,0.320419,1396.0,70.00,140.000000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False


In [83]:
kf = KFold(n_splits=5)#5개로 분할

In [84]:
X = data.drop('selling_price', axis = 1)
y = data['selling_price']

In [85]:
for i, j in kf.split(X):#5개의 인덱스가 나오게 됨, 분할을 하면 훈련셋과 시험셋으로 사용할 인덱스가 따로 나옴
  print(i,j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [86]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y.loc[train_index], y.loc[test_index]

In [87]:
train_rmse_total = []
test_rmse_total = []

In [88]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y.loc[train_index], y.loc[test_index]

  model = RandomForestRegressor(random_state = 100)
  model.fit(X_train, y_train)
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  train_rmse = mean_squared_error(y_train, train_pred)**0.5
  test_rmse = mean_squared_error(y_test, test_pred)**0.5

  train_rmse_total.append(train_rmse)
  test_rmse_total.append(test_rmse)

In [89]:
train_rmse_total

[50825.512918425346,
 58854.04054344074,
 57904.19615940739,
 56218.23740006373,
 58967.14500188981]

In [90]:
print("train_rmse:", sum(train_rmse_total)/5)#일반적인 성능

train_rmse: 56553.8264046454


#K-Fold 교차검증
##교차검증의 목적 : 교차 타당성
##- 모델의 예측력을 더 안정적으로 평가하기 위함
##- 새로운 데이터를 얼마나 잘 예측하는지 확인하고자 시험셋을 나누어서 평가함
##- 훈련셋, 시험셋 -> 데이터 분할은 랜덤하게 이루어지므로 어느정도 안정적인 보장을 받음

#K-Fold 교차검증의 아이디어
##- 데이터를 특정 개수(K개)로 쪼개어서 그중 하나씩을 선택하여 시험셋으로 사용하되, 이과정을 K번만큼 반복함

#하이퍼파라미터 튜닝(주로 사용하는 매개변수)
##n_estimators
##- 랜덤 포레스트를 구성하는 결정 트리의 개수
##- 기본값으 100으로 설정되어 있음
##- 너무 많거나 적은 수를 입력하면 성능이 떨어지므로 적정 수준의 값을 찾아서 넣어야 함

##max_depth
##- 결정 트리와 동일하게, 각 트리의 최대 깊이를 제한함
##- 숫자가 낮을수록 오버피팅을 피할 수 있으며, 또한 언더피팅의 위험도 올라감

##min_samples_slit
##- 해당 노드를 나눌 것인지 말 것인지를 노드 데이터 수를 기준으로 판단함
##- 이 매개변수에 지정된 숫자보다 적은 수의 데이터가 노드에 있으면 더는 분류하지 않음
##- 숫자가 높을수록 분리되는 노드가 적어질 것이므로, 오버피팅을 피하는 방법이자 언더피팅의 위험도 있음
##- 기본값은 2

##min_samples_leaf
##- 분리된 노드의 데이터에 최소 몇 개의 데이터가 있어야 할지를 결정하는 매개변수임
##- 여기에 지정된 숫자보다 적은 수의 데이터가 분류된다면, 해당 분리는 이루어지지 않음
##- 마찬가지로 숫자가 클수록 오버피팅을 피할 수 있고, 언더피팅의 위험도는 높아짐
##- 기본값은 1

##n-jobs
##- 병렬 처리에 사용되는 CPU 코어 수
##- 많으 코어를 사용할수록 속도가 빨라지며 -1을 입력하면 지원하는 모든 코어를 사용함
##- 기본값은 None으로, 실제는 1개 코어를 사용함
##- 랜덤 포레스트의 속도가 다소 느린만큼 충분한 코어를 사용하는 게 좋음

##1단계 : 문제정의 : 중고차의 가격을 예측하는 모델
##2단계 : 라이브러리 및 데이터 불러오기, 데이터 확인하기
##3단계 : 전처리
##ㄴ텍스트 데이터 : 수치로 활용하는 변수가 문자로 되어 있음
##ㄴ결측치 처리와 더미 변스 변환 : 숫자 단위로 분리, 결측치가 있는 행동을 제거하고 범주형 변수들을 더미 변수로 변환
##4단계 : 모델링 예측하기 : 랜덤 포레스트를 사용하여 예측 모델 만듦, 연속형 변수이기 대문에 아직 RMSE를 단적으로 '좋다/나쁘다' 평가할 수 없음
##5단계 : 이해하기
##ㄴK겹 교차검증 : 시험셋의 다양한 조합으로 신뢰할만한 결과를 얻은 검증을 학습
##ㄴ랜덤 포레스트 : 랜덤 포레스트는 결정트릐의 진화된 버전으로, 여러 독립된 결정 트리들이 합심하여 예측
##6단계 : 하이퍼 파라미터 튜닝 ; 랜덤 포레스트에서 쓸 수 있는 몇 가지 하이퍼 파라미터 변경 -> test rmse(평균 제곱근 편차)를 조금 낮춤